In [1]:
import numpy
import requests
import pandas as pd

In [2]:
job_postings = pd.read_csv('linkedin_job_postings.csv')
job_postings.head()


,job_link,last_processed_time,got_summary,got_ner,is_being_worked,job_title,company,job_location,first_seen,search_city,search_country,search_position,job_level,job_type
0,https://www.linkedin.com/jobs/view/account-exe...,2024-01-21 07:12:29.00256+00,t,t,f,Account Executive - Dispensing (NorCal/Norther...,BD,"San Diego, CA",2024-01-15,Coronado,United States,Color Maker,Mid senior,Onsite
1,https://www.linkedin.com/jobs/view/registered-...,2024-01-21 07:39:58.88137+00,t,t,f,Registered Nurse - RN Care Manager,Trinity Health MI,"Norton Shores, MI",2024-01-14,Grand Haven,United States,Director Nursing Service,Mid senior,Onsite
2,https://www.linkedin.com/jobs/view/restaurant-...,2024-01-21 07:40:00.251126+00,t,t,f,RESTAURANT SUPERVISOR - THE FORKLIFT,Wasatch Adaptive Sports,"Sandy, UT",2024-01-14,Tooele,United States,Stand-In,Mid senior,Onsite
3,https://www.linkedin.com/jobs/view/independent...,2024-01-21 07:40:00.308133+00,t,t,f,Independent Real Estate Agent,Howard Hanna | Rand Realty,"Englewood Cliffs, NJ",2024-01-16,Pinehurst,United States,Real-Estate Clerk,Mid senior,Onsite
4,https://www.linkedin.com/jobs/view/group-unit-...,2024-01-19 09:45:09.215838+00,f,f,f,Group/Unit Supervisor (Systems Support Manager...,"IRS, Office of Chief Counsel","Chamblee, GA",2024-01-17,Gadsden,United States,Supervisor Travel-Information Center,Mid senior,Onsite


In [3]:

job_skills = pd.read_csv('job_skills.csv')
job_skills.head()

,job_link,job_skills
0,https://www.linkedin.com/jobs/view/housekeeper...,"Building Custodial Services, Cleaning, Janitor..."
1,https://www.linkedin.com/jobs/view/assistant-g...,"Customer service, Restaurant management, Food ..."
2,https://www.linkedin.com/jobs/view/school-base...,"Applied Behavior Analysis (ABA), Data analysis..."
3,https://www.linkedin.com/jobs/view/electrical-...,"Electrical Engineering, Project Controls, Sche..."
4,https://www.linkedin.com/jobs/view/electrical-...,"Electrical Assembly, Point to point wiring, St..."


In [4]:
df_merged = pd.merge(job_skills, job_postings[['job_link', 'job_title', 'company', 'job_level']], on='job_link', how='inner')
df_merged.head(30)

,job_link,job_skills,job_title,company,job_level
0,https://www.linkedin.com/jobs/view/housekeeper...,"Building Custodial Services, Cleaning, Janitor...",Housekeeper I - PT,Jacksonville State University,Mid senior
1,https://www.linkedin.com/jobs/view/assistant-g...,"Customer service, Restaurant management, Food ...",Assistant General Manager - Huntington 4131,Ruby Tuesday,Mid senior
2,https://www.linkedin.com/jobs/view/school-base...,"Applied Behavior Analysis (ABA), Data analysis...",School-based Behavior Analyst,CCRES Educational and Behavioral Health Services,Mid senior
3,https://www.linkedin.com/jobs/view/electrical-...,"Electrical Engineering, Project Controls, Sche...",Electrical Deputy Engineering Group Supervisor,Energy Jobline,Mid senior
4,https://www.linkedin.com/jobs/view/electrical-...,"Electrical Assembly, Point to point wiring, St...",Electrical Assembly Lead,Sanmina,Mid senior
5,https://www.linkedin.com/jobs/view/senior-lead...,"Access Control, Video Management Systems, IPba...",Senior Lead Technician/Programmer,Security 101,Mid senior
6,https://www.linkedin.com/jobs/view/program-con...,"Consultation, Supervision, InService Training,...",Program Consultant,Methodist Family Health,Mid senior
7,https://www.linkedin.com/jobs/view/veterinary-...,"Veterinary Receptionist, AAHAaccredited, Custo...",Veterinary Receptionist,WellHaven Pet Health,Mid senior
8,https://www.linkedin.com/jobs/view/sr-technici...,"Optical Inspection Equipment Programming, MS E...","Sr. Technician, Receiving & Inspection",Abbott,Associate
9,https://www.linkedin.com/jobs/view/experienced...,"HVAC, troubleshooting, Preventative maintenanc...",Experienced HVAC Service Technician,Lane Valente Industries,Mid senior


In [5]:
df_merged_sample = df_merged.sample(frac=0.02, random_state=42)
df_merged_sample.to_csv('merged_jobset_sample.csv')